In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# import tensorflow_datasets as tfds

: 

: 

## Loading the dataset

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def train():
    pass